In [19]:
import pickle
c2vpath = 'char2vec.model'
c2vfile = open(c2vpath, 'rb')
cdict = pickle.load(c2vfile)
emat = pickle.load(c2vfile)
c2vfile.close()

In [2]:
import pandas as pd
df_1 = pd.read_csv('atec_nlp_sim_train.csv', sep ='\t', encoding='utf8')
df_2 = pd.read_csv('atec_nlp_sim_train_add.csv', sep ='\t', encoding='utf8')
df = df_1.append(df_2)
print(df[:5])

In [17]:
def readdata(paths, mode='train'):
    data = []
    for path in paths:
        with open(path, 'r', encoding='utf8') as f:
            line = f.readline().strip()
            if len(line) > 0:
                line = line.split('\t')
                q1 = line[1]
                q2 = line[2]
                if mode == 'train':
                    lb = line[-1]
                    traindata.append((q1, q2, lb))
                else:
                    traindata.append((q1, q2))
    return data
paths = ['atec_nlp_sim_train.csv', 'atec_nlp_sim_train_add.csv']
TRAINDATA_RAW = readdata(paths)

In [11]:
maxlen = lambda x: max(max(len(i[0]), len(i[1])) for i in x)
print(maxlen(traindata))

30


In [16]:
import numpy as np
def sent2int(sent, cdict, maxlen=30):
    result = [0]*maxlen
    for i in range(min(maxlen, len(sent))):
        if sent[i] in cdict:
            result[i] = cdict[sent[i]]
        else:
            result[i] = cdict['OOV']
    result = np.array(result)
    return result

s = u'花呗开通年级'
print(sent2int(s, cdict))

[  920 45962   102   228   285  1930     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


In [18]:
import keras

def prepare(rawdata, mode='train'):
    result = []
    for smp in rawdata:
        q1_vec = sent2int(smp[0])
        q2_vec = sent2int(smp[1])
        if mode == 'train':
            label = int(smp[-1])
            result.append((q1_vec, q2_vec, label))
        else:
            result.append((q1_vec, q2_vec))
    return result

traindata = prepare(TRAINDATA_RAW)

/home/faust/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/faust/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
import random
random.shuffle(traindata)

train_split = traindata[:int(len(traindata)*0.9)]
val_split = traindata[int(len(traindata)*0.9):]

# MODEL

In [23]:
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv1D, TimeDistributed
from keras.layers import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D, GlobalAveragePooling1D
from keras.layers import Dropout, BatchNormalization
from keras.layers import concatenate, Flatten, regularizers
from keras.layers import Add, Subtract, Multiply, Concatenate, Reshape
from keras.layers import Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu, sigmoid, tanh
from keras import backend as K
from keras import optimizers


MAXLEN = 30
VOCABLEN = len(cdict)
EMBEDDING_DIM = 200
DROPOUT = 0.5

input_q1 = Input(shape=(MAXLEN,))
input_q2 = Input(shape=(MAXLEN,))

layer_embedding_c = Embedding(VOCABLEN,
                              EMBEDDING_DIM,
                              weights=[emat],
                              input_length=MAXLEN,
                              trainable=False)

filter_size = [1, 2, 3, 5]
layer_conv_c1 = [Conv1D(64, size, activation='relu', padding='same') for size in filter_size]  # char conv1
conv_1_pool = [MaxPooling1D(ws) for ws in [3, 5, 7]]
layer_fc_c1 = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))  # char fc1

layer_conv_c2 = [Conv1D(64, size, activation='relu', padding='same') for size in filter_size]  # char conv2
conv_2_pool = [MaxPooling1D(ws) for ws in [7, 9, 11]]
layer_fc_c2 = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))  # char fc2
layer_conv_c3 = [Conv1D(64, size, activation='relu', padding='same') for size in filter_size]  # char conv3

# ------ embedding -------
q1_embed_c = layer_embedding_c(input_q1)
q2_embed_c = layer_embedding_c(input_q2)

# ------ conv 1 -------
q1_embed_c = BatchNormalization()(q1_embed_c)
q2_embed_c = BatchNormalization()(q2_embed_c)
q1_embed_c = Dropout(0.2)(q1_embed_c)
q2_embed_c = Dropout(0.2)(q2_embed_c)

q1_conv_c1 = [conv(q1_embed_c) for conv in layer_conv_c1]
q1_conv_c1 = Concatenate()(q1_conv_c1)
q2_conv_c1 = [conv(q2_embed_c) for conv in layer_conv_c1]
q2_conv_c1 = Concatenate()(q2_conv_c1)

q1_pool_c1 = [Flatten()(mp(q1_conv_c1)) for mp in conv_1_pool]
q1_pool_c1 = Concatenate()(q1_pool_c1)
q2_pool_c1 = [Flatten()(mp(q2_conv_c1)) for mp in conv_1_pool]
q2_pool_c1 = Concatenate()(q2_pool_c1)

pair_sub_c1 = Subtract()([layer_fc_c1(q1_pool_c1), layer_fc_c1(q2_pool_c1)])
pair_feat_c1 = Lambda(lambda x: x ** 2)(pair_sub_c1)

# # ------ conv 2 -------
q1_conv_c1 = BatchNormalization()(q1_conv_c1)
q2_conv_c1 = BatchNormalization()(q2_conv_c1)
q1_conv_c1 = Dropout(0.5)(q1_conv_c1)
q2_conv_c1 = Dropout(0.5)(q2_conv_c1)

q1_conv_c2 = [conv(q1_conv_c1) for conv in layer_conv_c2]
q1_conv_c2 = Concatenate()(q1_conv_c2)
q2_conv_c2 = [conv(q2_conv_c1) for conv in layer_conv_c2]
q2_conv_c2 = Concatenate()(q2_conv_c2)

q1_pool_c2 = [Flatten()(mp(q1_conv_c2)) for mp in conv_2_pool]
q1_pool_c2 = Concatenate()(q1_pool_c2)
q2_pool_c2 = [Flatten()(mp(q2_conv_c2)) for mp in conv_2_pool]
q2_pool_c2 = Concatenate()(q2_pool_c2)

pair_sub_c2 = Subtract()([layer_fc_c2(q1_pool_c2), layer_fc_c2(q2_pool_c2)])
pair_feat_c2 = Lambda(lambda x: x ** 2)(pair_sub_c2)

# # ------ conv 3 -------
q1_conv_c2 = BatchNormalization()(q1_conv_c2)
q2_conv_c2 = BatchNormalization()(q2_conv_c2)
q1_conv_c2 = Dropout(0.5)(q1_conv_c2)
q2_conv_c2 = Dropout(0.5)(q2_conv_c2)

q1_conv_c3 = [conv(q1_conv_c2) for conv in layer_conv_c3]
q1_conv_c3 = Concatenate()(q1_conv_c3)
q2_conv_c3 = [conv(q2_conv_c2) for conv in layer_conv_c3]
q2_conv_c3 = Concatenate()(q2_conv_c3)

q1_pool_c3 = GlobalMaxPooling1D()(q1_conv_c3)
q2_pool_c3 = GlobalMaxPooling1D()(q2_conv_c3)

pair_sub_c3 = Subtract()([q1_pool_c3, q2_pool_c3])
pair_feat_c3 = Lambda(lambda x: x ** 2)(pair_sub_c3)

# # ------ concat all -------
pair = Concatenate()([pair_feat_c1, pair_feat_c2, pair_feat_c3])
pair = BatchNormalization()(pair)
pair = Dropout(0.5)(pair)

# ------ last hidden -------
pair_last = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.1))(pair)
# pair_last = Dense(512, activation='tanh')(pair)
# pair_last = Dense(128, activation='tanh')(pair_last)

# ------ predict layer -------
pair_last = BatchNormalization()(pair_last)
pair_last = Dropout(0.5)(pair_last)
predict = Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.1))(pair_last)

# ------ build model -------
model = Model(inputs=[input_q1, input_q2], outputs=predict)

# ------ optimizer -------
LEARNING_RATE = 0.0025
# optimizer = optimizers.SGD(lr=0.001, decay=1e-5, momentum=0.5, nesterov=True)
# optimizer = optimizers.Adagrad(lr=LEARNING_RATE, epsilon=1e-06)
# optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)
optimizer = optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.compile(loss=focal_loss,
#               optimizer=optimizer,
#               metrics=['binary_crossentropy'])
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['binary_crossentropy'])

ResourceExhaustedError: OOM when allocating tensor with shape[175818,200]
	 [[Node: embedding_4/embeddings/Assign = Assign[T=DT_FLOAT, _class=["loc:@embedding_4/embeddings"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_4/embeddings, embedding_4/random_uniform)]]

Caused by op 'embedding_4/embeddings/Assign', defined at:
  File "/home/faust/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/faust/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/faust/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/faust/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/faust/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
 

# TRAIN

# TEST